In [ ]:
 !R


> ^C


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import keras
from PIL import Image
from keras.datasets import fashion_mnist
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

In [ ]:
names = ["Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
         "Naive Bayes", "QDA"]

In [ ]:
classifiers = [
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1, max_iter=1000),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()]

In [ ]:
def add_noise(img):
    '''Add random noise to an image'''
    VARIABILITY = 10
    deviation = VARIABILITY*random.random()
    noise = np.random.normal(0, deviation, img.shape)
    img += noise
    np.clip(img, 0., 255.)
    return img

In [ ]:
def GenerateDatanoise(X_train_idx):
  # Prepare data-augmenting data generator
  from keras.preprocessing.image import ImageDataGenerator
  datagen = ImageDataGenerator(
          rescale=1./255,
          rotation_range=20,
          width_shift_range=0.1,
          height_shift_range=0.1,
          zoom_range=0.1,
          preprocessing_function=add_noise,
      )
  # Load a single image as our example
  from keras.preprocessing import image
  img = X_train_idx

  # Generate distorted images
  images = [img]
  img_arr = image.img_to_array(img)
  img_arr = img_arr.reshape((1,) + img_arr.shape)
  for batch in datagen.flow(img_arr, batch_size=1):
      images.append( image.array_to_img(batch[0]) )
      if len(images) >= 1000:
          break
  return images

In [ ]:
def Aumentarexemplos(X_train, y_train, X_test, y_test):
  idxmais = [5397]
  for j in idxmais:
    #idxmais = 5   
    datanoise = GenerateDatanoise(X_test[j])
    for i in range(0, len(datanoise)):
      X_train = np.append(X_train, [np.array(datanoise[i])], axis=0)
      y_train = np.append(y_train, y_train[j])

  print("X_train shape", X_train.shape)
  print("y_train shape", y_train.shape)
  print("X_test shape", X_test.shape)
  print("y_test shape", y_test.shape)

  return (X_train, y_train), (X_test, y_test)

In [ ]:
def Reduzirclasses(class1, class2, X_train, y_train, X_test, y_test):
  for digit in range(0,10):
    if (digit != class1) and (digit != class2):
      ind_class = np.where(y_train == digit)
      y_train = np.delete(y_train, ind_class)
      X_train = np.delete(X_train, ind_class, axis = 0)

      ind_class_test = np.where(y_test == digit)
      y_test = np.delete(y_test, ind_class_test)
      X_test = np.delete(X_test, ind_class_test, axis = 0)

  print("X_train shape", X_train.shape)
  print("y_train shape", y_train.shape)
  print("X_test shape", X_test.shape)
  print("y_test shape", y_test.shape)

  return (X_train, y_train), (X_test, y_test) 

In [ ]:
(train_images, train_labels), (test_images, test_labels)  = fashion_mnist.load_data()
#Exemplos Distantes
#class1 = 2
#class2 = 7
#Exemplos Próximos
class1 = 2
class2 = 4

#(train_images, train_labels), (test_images, test_labels)   = Reduzirclasses(class1, class2, train_images, train_labels, test_images, test_labels)

(train_images, train_labels), (test_images, test_labels)  = Aumentarexemplos(train_images, train_labels, test_images, test_labels)

n_X_train = len(train_images)
n_X_test = len(test_images)

train_images = train_images.reshape(n_X_train, 784)
test_images = test_images.reshape(n_X_test, 784)

X_train shape (61000, 28, 28)
y_train shape (61000,)
X_test shape (10000, 28, 28)
y_test shape (10000,)


In [ ]:
scores = []
for name, clf in zip(names, classifiers):
  clf.fit(train_images, train_labels)
  score = clf.score(test_images, test_labels)
  print("Score ", name, ": ", score)
  scores.append(score)

Score  Decision Tree :  0.6944
Score  Random Forest :  0.6985
Score  Neural Net :  0.8514
Score  AdaBoost :  0.5676
Score  Naive Bayes :  0.5639


/usr/local/lib/python3.7/dist-packages/sklearn/discriminant_analysis.py:691: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


Score  QDA :  0.5677


In [ ]:
pred1 = classifiers[0].predict(test_images)
pred2 = classifiers[1].predict(test_images)
pred3 = classifiers[2].predict(test_images)
pred4 = classifiers[3].predict(test_images)
pred5 = classifiers[4].predict(test_images)
pred6 = classifiers[5].predict(test_images)

predictions = [pred1, pred2, pred3, pred4, pred5, pred6]

In [ ]:
#Escolha aleatória de 100 imagens para cada label. (PARA 10 CLASSES)
ind_random =[]
for digit in range(0, 10):
  ind_class = np.where(test_labels == digit)
  np.random.shuffle(ind_class[0])
  ind_random.append(ind_class[0][1:101])

idx = []
num_classes = 10
for i in range(num_classes):
  for j in range(100):
    idx.append(ind_random[i][j])
print(idx)
print('a')

[8197, 1483, 1203, 7158, 7787, 3115, 2420, 9815, 4690, 8350, 2524, 4424, 951, 7626, 9764, 7127, 2968, 3729, 8029, 4206, 8948, 5129, 2872, 7749, 2638, 3557, 6847, 9748, 6972, 1572, 2829, 7774, 9508, 2374, 7940, 858, 9686, 1169, 8815, 2308, 3831, 6293, 9371, 7413, 9809, 7864, 2512, 7824, 6929, 2286, 965, 2047, 2652, 4818, 8564, 6225, 5296, 8851, 8689, 3065, 7623, 8812, 9494, 2506, 1000, 8091, 1948, 3356, 910, 5876, 9061, 6074, 6760, 8031, 7117, 7585, 636, 889, 6065, 1270, 8394, 9021, 6572, 5307, 577, 8657, 2393, 5577, 1403, 5718, 3819, 4121, 9981, 5246, 8168, 8324, 275, 3277, 374, 9342, 7744, 2771, 8535, 2254, 2342, 2643, 3965, 9310, 4974, 4780, 8400, 2729, 4321, 216, 2534, 7150, 1242, 604, 3840, 6125, 6319, 334, 179, 8768, 4452, 7423, 5127, 5812, 6117, 2770, 6053, 2612, 3627, 3597, 2835, 7666, 1664, 3654, 1319, 6336, 5175, 9994, 2640, 4633, 6401, 6100, 8679, 9818, 7808, 9103, 3079, 8539, 9666, 6935, 7693, 5753, 6045, 886, 6080, 8107, 6974, 3000, 4476, 1800, 4482, 2497, 8164, 8200, 660, 

In [ ]:
#Escolha aleatória de 100 imagens para cada label. (PARA 2 CLASSES)
ind_random =[]
labels_distantes = [2, 7]
labels_proximos = [2, 4]
for digit in labels_proximos:
  ind_class = np.where(test_labels == digit)
  np.random.shuffle(ind_class[0])
  ind_random.append(ind_class[0][1:101])

idx = []
num_classes = 2
for i in range(num_classes):
  for j in range(100):
   idx.append(ind_random[i][j])
print(idx)
print(len(idx))

In [ ]:
idx= [8197, 1483, 1203, 7158, 7787, 3115, 2420, 9815, 4690, 8350, 2524, 4424, 951, 7626, 9764, 7127, 2968, 3729, 8029, 4206, 8948, 5129, 2872, 7749, 2638, 3557, 6847, 9748, 6972, 1572, 2829, 7774, 9508, 2374, 7940, 858, 9686, 1169, 8815, 2308, 3831, 6293, 9371, 7413, 9809, 7864, 2512, 7824, 6929, 2286, 965, 2047, 2652, 4818, 8564, 6225, 5296, 8851, 8689, 3065, 7623, 8812, 9494, 2506, 1000, 8091, 1948, 3356, 910, 5876, 9061, 6074, 6760, 8031, 7117, 7585, 636, 889, 6065, 1270, 8394, 9021, 6572, 5307, 577, 8657, 2393, 5577, 1403, 5718, 3819, 4121, 9981, 5246, 8168, 8324, 275, 3277, 374, 9342, 7744, 2771, 8535, 2254, 2342, 2643, 3965, 9310, 4974, 4780, 8400, 2729, 4321, 216, 2534, 7150, 1242, 604, 3840, 6125, 6319, 334, 179, 8768, 4452, 7423, 5127, 5812, 6117, 2770, 6053, 2612, 3627, 3597, 2835, 7666, 1664, 3654, 1319, 6336, 5175, 9994, 2640, 4633, 6401, 6100, 8679, 9818, 7808, 9103, 3079, 8539, 9666, 6935, 7693, 5753, 6045, 886, 6080, 8107, 6974, 3000, 4476, 1800, 4482, 2497, 8164, 8200, 660, 5323, 4008, 849, 7920, 8156, 5538, 7184, 2518, 5545, 6415, 2414, 9525, 8228, 4630, 8908, 3561, 7276, 1860, 8064, 9098, 5845, 6763, 8133, 5639, 1219, 3243, 6161, 5843, 4445, 8861, 8053, 4091, 7926, 2208, 4847, 5383, 7810, 4239, 3288, 5713, 4588, 8662, 9784, 3918, 7490, 8498, 5751, 2030, 7452, 3278, 4671, 2547, 16, 9881, 6625, 3726, 9600, 2315, 3164, 5715, 5125, 2601, 7230, 1533, 255, 6885, 4203, 7958, 1361, 9329, 5820, 608, 6243, 1538, 9172, 5647, 9604, 4160, 7283, 2977, 4260, 5250, 668, 9259, 4363, 2908, 2650, 3563, 841, 8500, 3255, 1661, 6459, 4598, 4037, 1042, 1279, 7639, 3365, 9706, 8595, 7753, 6852, 72, 5775, 3394, 2272, 5696, 1926, 4873, 4323, 6237, 4255, 367, 8912, 4196, 5527, 1192, 2337, 457, 4465, 6627, 7633, 6486, 6439, 9746, 8362, 2636, 5075, 2081, 6291, 6583, 931, 9225, 5610, 3722, 9838, 8983, 3833, 9271, 7565, 8928, 3973, 9102, 5664, 7635, 4084, 91, 1801, 271, 2688, 8008, 4491, 7769, 7267, 1920, 4005, 8896, 2312, 9554, 7827, 2256, 9260, 7429, 4915, 9984, 8836, 1631, 3407, 539, 5085, 3007, 7088, 1653, 7681, 466, 1150, 4415, 8864, 2666, 3945, 6748, 376, 2715, 7364, 1968, 1489, 5361, 1047, 2475, 5145, 4435, 4544, 8141, 6354, 698, 1834, 5967, 4667, 3936, 6377, 6907, 1057, 1897, 8377, 5562, 1022, 7264, 6154, 450, 5855, 7587, 8741, 7405, 7415, 6891, 1844, 3795, 1311, 5179, 2844, 7628, 1585, 2695, 2784, 7322, 6423, 8489, 2012, 3616, 1001, 3584, 540, 8140, 4905, 5496, 7594, 1879, 967, 5310, 2462, 1792, 649, 1679, 279, 2737, 9496, 4417, 4464, 2671, 4393, 6751, 1385, 9062, 6908, 6148, 875, 3672, 5774, 2679, 57, 9702, 9643, 6801, 363, 168, 1074, 928, 7642, 2681, 8944, 9040, 10, 6837, 2901, 210, 2273, 153, 3195, 8100, 432, 987, 4095, 1185, 9750, 1077, 1251, 5993, 4373, 3849, 2150, 1132, 1312, 935, 4627, 4096, 2992, 8580, 3430, 3811, 2244, 7965, 637, 3447, 8712, 8891, 1142, 7228, 5124, 5583, 1050, 5397, 3295, 7736, 8104, 8330, 8730, 7110, 8116, 3643, 7991, 4589, 6270, 1362, 7986, 1172, 4739, 6500, 4995, 1075, 1611, 3566, 2103, 1325, 2714, 4171, 8113, 4815, 1092, 3662, 7484, 9213, 8026, 8463, 9760, 7293, 6947, 7768, 6147, 323, 3696, 1776, 7861, 292, 9216, 7027, 6244, 5215, 5290, 8230, 6550, 9374, 1770, 5681, 6945, 690, 7380, 6123, 9629, 8419, 5431, 3893, 502, 7889, 7776, 3618, 1727, 9324, 2183, 7409, 5152, 9941, 2238, 6060, 288, 5451, 3547, 2833, 654, 7074, 7197, 6634, 4843, 3200, 7844, 535, 7206, 6791, 4360, 6194, 9922, 4181, 6979, 4717, 4022, 4076, 4090, 8170, 9375, 9754, 4254, 1435, 2305, 2223, 5781, 1829, 9581, 2748, 2080, 8473, 4888, 8103, 3594, 3645, 5703, 7867, 852, 8240, 1637, 6727, 9099, 5569, 6364, 3165, 684, 3959, 9868, 8940, 9146, 9057, 1940, 9961, 4994, 2009, 5654, 285, 7112, 6322, 4677, 7111, 5410, 3020, 9513, 1277, 4909, 7881, 7012, 7962, 344, 622, 9684, 5990, 9573, 7840, 5452, 3203, 761, 4420, 8995, 9094, 634, 716, 1174, 7903, 966, 322, 226, 5052, 9957, 2459, 2588, 7764, 725, 4152, 7538, 2551, 7296, 4083, 1796, 7004, 5169, 7258, 8769, 8974, 8349, 8664, 2708, 9097, 5571, 6876, 8005, 2247, 7222, 5977, 3257, 7024, 561, 3724, 6663, 193, 3470, 5254, 4941, 5589, 8658, 4082, 9487, 9004, 3347, 1756, 6384, 6206, 8538, 1629, 7303, 9206, 6540, 9831, 9690, 5329, 5378, 4580, 9125, 4634, 230, 2410, 712, 9401, 9197, 3949, 6686, 7783, 3550, 6385, 927, 5050, 4681, 9262, 3498, 3825, 503, 351, 8817, 6779, 1382, 5723, 6792, 7580, 8181, 5045, 1795, 8562, 4773, 4101, 336, 6698, 7923, 1323, 8808, 7759, 5414, 4528, 5944, 7120, 7919, 3030, 5988, 3274, 6490, 4560, 3630, 8493, 6713, 5447, 8520, 2242, 7885, 9860, 8318, 5178, 5861, 6431, 7242, 9044, 5302, 5208, 6209, 402, 7960, 6836, 6534, 6680, 6719, 6677, 1534, 3378, 5353, 43, 8391, 4117, 4330, 8325, 9144, 3538, 7728, 8553, 4999, 6814, 901, 9266, 5868, 1915, 6482, 3136, 8700, 6541, 1956, 5846, 2077, 1089, 3559, 2304, 3468, 5784, 3189, 4341, 4805, 6443, 2389, 9301, 7168, 5838, 3709, 6536, 5403, 6410, 8287, 5147, 8844, 5390, 9022, 2561, 9007, 3376, 7689, 5376, 9321, 2000, 7340, 1737, 4844, 1455, 7513, 5717, 8070, 2799, 429, 9108, 4232, 8904, 6930, 2947, 929, 9226, 8446, 882, 8321, 1751, 1126, 9549, 1406, 3242, 7091, 220, 9741, 5122, 8547, 1470, 7480, 8188, 9992, 1884, 4989, 6522, 8286, 424, 533, 126, 3757, 5658, 529, 5430, 2557, 3814, 4026, 5402, 6403, 2005, 1845, 1846, 4436, 5217, 3354, 9234, 5470, 9096, 116, 9642, 1427, 9923, 642, 7927, 7269, 1715, 6421, 3869, 9636, 2027, 3725, 9533, 3680, 7860, 268, 5101, 3314, 8504, 6359, 5077, 7644, 6984, 2520, 3721, 4296, 9370, 8922, 9759, 980, 1133, 4614, 5890, 7658, 3395, 7010, 9915, 2250, 556, 6769, 5009, 1709, 5989, 5560, 810, 8460, 1141, 6029, 5439, 4830, 7113, 9083, 2494, 1962, 1418, 5553, 122, 3308, 5891, 1068, 9058, 9739, 9139, 2187, 6017, 4997, 7145, 2488, 8398, 6733, 2768, 1894, 680, 9053, 5111, 4044, 2113, 3834, 7865, 5975, 6537, 5600, 9757, 9978, 4631, 6267, 4450, 5019, 4002, 2056, 3477, 9175, 3408, 7044, 7902, 2935, 7099, 1505, 5938, 1522, 6552, 9526, 23, 683, 7378, 9614, 6005, 1544, 4369, 8479, 5314, 9964, 5102, 6130]

In [ ]:
#Escolha aleatória de 100 imagens para cada label (2 classes = 200 idx)
#distantes
#idx = [682, 517, 427, 89, 501, 1593, 279, 1585, 1044, 433, 129, 1245, 580, 618, 1270, 347, 1558, 1799, 564, 269, 838, 1146, 944, 1827, 1844, 689, 1198, 97, 270, 1553, 1111, 213, 823, 382, 110, 1865, 1417, 1936, 1986, 885, 1670, 669, 907, 1135, 1744, 271, 83, 63, 995, 755, 34, 1058, 929, 1179, 496, 1391, 1649, 709, 1796, 1447, 692, 1780, 226, 1763, 1579, 622, 1390, 1345, 1628, 581, 1143, 1847, 118, 1898, 510, 1186, 411, 1038, 1067, 441, 95, 491, 1998, 1990, 196, 1250, 1830, 1342, 749, 1672, 432, 1458, 404, 146, 1671, 1532, 357, 1164, 531, 650, 381, 702, 1551, 543, 690, 900, 1088, 539, 420, 1712, 557, 1057, 163, 352, 1846, 839, 582, 1029, 1282, 1610, 1683, 284, 332, 1149, 635, 350, 1769, 323, 1838, 16, 1008, 326, 455, 1448, 1915, 1828, 1414, 85, 1906, 154, 1061, 189, 1274, 520, 1337, 545, 416, 1225, 977, 1520, 1662, 1682, 1788, 1123, 1573, 1354, 1352, 1094, 1297, 1110, 1705, 452, 69, 1228, 745, 1639, 1389, 1816, 533, 911, 397, 1176, 1924, 1582, 487, 1702, 1372, 406, 731, 760, 976, 159, 1789, 318, 619, 1247, 145, 1782, 243, 1633, 1891, 139, 558, 762, 1836, 903, 688, 563, 1962, 846]
#próximos
idx = [725, 1683, 1424, 1708, 1334, 1930, 122, 1776, 1446, 1707, 415, 54, 1737, 445, 278, 885, 1579, 1780, 1063, 674, 1544, 1727, 1862, 745, 1565, 625, 349, 366, 479, 1439, 1154, 1794, 855, 630, 1413, 1427, 1281, 323, 1060, 1920, 1554, 901, 1891, 1169, 487, 1019, 1445, 1151, 452, 917, 528, 1758, 722, 1848, 539, 1403, 1101, 306, 1346, 1201, 1071, 1186, 1469, 1261, 384, 347, 652, 1716, 1000, 1495, 558, 10, 762, 667, 1487, 1481, 886, 1606, 1752, 453, 1310, 1393, 1351, 1476, 875, 1355, 56, 1950, 365, 1454, 1957, 1945, 1142, 657, 1566, 663, 668, 1893, 1237, 636, 1079, 899, 1985, 39, 1724, 533, 936, 1983, 260, 1250, 1400, 1248, 1688, 593, 483, 591, 1798, 1881, 1179, 971, 1466, 1200, 1618, 1593, 345, 397, 631, 906, 1592, 1236, 1022, 1027, 575, 847, 1615, 190, 817, 423, 1141, 1116, 687, 570, 1189, 1190, 500, 336, 371, 22, 1590, 545, 499, 140, 627, 916, 1388, 85, 1329, 523, 1373, 1150, 460, 289, 502, 135, 1754, 1651, 1024, 276, 1354, 793, 1702, 1885, 1067, 1382, 409, 717, 585, 1028, 35, 1068, 1490, 1110, 951, 372, 938, 747, 1807, 1280, 1091, 253, 1499, 1399, 1489, 1340, 442, 800, 1577, 1946, 1879, 1402]


In [ ]:
resultado = []
for pred in predictions:
  labels = []
  preds = []
  for i in idx: 
    labels.append(test_labels[i])
    preds.append(pred[i])
  res = []
  for i, j in zip(labels, preds):
    if(i == j):
      z = 1
    else:
      z = 0
    res.append(z)
  resultado.append(res)
print(resultado[4])

[0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 

In [ ]:
colunas = []
nitens = 1000 
for i in range(1, nitens+1):
  colunas.append("Item."+ str(i))
print(colunas)

['Item.1', 'Item.2', 'Item.3', 'Item.4', 'Item.5', 'Item.6', 'Item.7', 'Item.8', 'Item.9', 'Item.10', 'Item.11', 'Item.12', 'Item.13', 'Item.14', 'Item.15', 'Item.16', 'Item.17', 'Item.18', 'Item.19', 'Item.20', 'Item.21', 'Item.22', 'Item.23', 'Item.24', 'Item.25', 'Item.26', 'Item.27', 'Item.28', 'Item.29', 'Item.30', 'Item.31', 'Item.32', 'Item.33', 'Item.34', 'Item.35', 'Item.36', 'Item.37', 'Item.38', 'Item.39', 'Item.40', 'Item.41', 'Item.42', 'Item.43', 'Item.44', 'Item.45', 'Item.46', 'Item.47', 'Item.48', 'Item.49', 'Item.50', 'Item.51', 'Item.52', 'Item.53', 'Item.54', 'Item.55', 'Item.56', 'Item.57', 'Item.58', 'Item.59', 'Item.60', 'Item.61', 'Item.62', 'Item.63', 'Item.64', 'Item.65', 'Item.66', 'Item.67', 'Item.68', 'Item.69', 'Item.70', 'Item.71', 'Item.72', 'Item.73', 'Item.74', 'Item.75', 'Item.76', 'Item.77', 'Item.78', 'Item.79', 'Item.80', 'Item.81', 'Item.82', 'Item.83', 'Item.84', 'Item.85', 'Item.86', 'Item.87', 'Item.88', 'Item.89', 'Item.90', 'Item.91', 'Item.9

In [ ]:
data=[] #certo(1), errado(0)
for i in resultado:
  data.append(i)

df = pd.DataFrame(data, columns = colunas)
df.loc[6]=[0]*nitens
df.loc[7]=[1]*nitens
print(df)
df.to_csv('/content/data.csv', index=False)

   Item.1  Item.2  Item.3  Item.4  ...  Item.997  Item.998  Item.999  Item.1000
0       0       0       0       1  ...         1         1         1          1
1       1       0       1       1  ...         1         1         1          1
2       1       1       1       1  ...         1         1         1          1
3       0       0       0       0  ...         0         0         0          1
4       0       0       0       0  ...         0         0         1          1
5       0       1       0       1  ...         1         1         1          1
6       0       0       0       0  ...         0         0         0          0
7       1       1       1       1  ...         1         1         1          1

[8 rows x 1000 columns]


In [ ]:
!Rscript tri.txt 

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)
trying URL 'https://cran.rstudio.com/src/contrib/devtools_2.4.2.tar.gz'
Content type 'application/x-gzip' length 371298 bytes (362 KB)
downloaded 362 KB

* installing *source* package ‘devtools’ ...
** package ‘devtools’ successfully unpacked and MD5 sums checked
** using staged installation
** R
** inst
** byte-compile and prepare package for lazy loading
** help
*** installing help indices
*** copying figures
** building package indices
** installing vignettes
** testing if installed package can be loaded from temporary location
** testing if installed package can be loaded from final location
** testing if installed package keeps a record of temporary installation path
* DONE (devtools)

The downloaded source packages are in
	‘/tmp/RtmpXQavcO/downloaded_packages’
Loading required package: usethis
Skipping install of 'mirt' from a github remote, the SHA1 (3a6af5e7) has not changed since last install.
  

In [ ]:
coef = pd.read_csv('/content/coef.csv')

In [ ]:
def formatar(coef): 
  nitens = 1000
  ncoef = 4
  totalvalues = nitens * ncoef
  
  a = []
  for i in range(0, totalvalues, 4):
    a.append(coef.iloc[i][1]) 
  b = []
  for i in range(1, totalvalues, 4):
    b.append(coef.iloc[i][1]) 
  c = []
  for i in range(2, totalvalues, 4):
    c.append(coef.iloc[i][1])
  d = [] 
  for i in range(3, totalvalues, 4):
    d.append(coef.iloc[i][1]) 

  df = pd.DataFrame({'itens': range(1, nitens + 1 ),
                   'a': a,
                   'b': b,
                   'c': c,
                   'd': d
                   })
  return df

In [ ]:
df = formatar(coef)

In [ ]:
df.to_csv('/content/idx.csv', index=False)

In [ ]:
dfa_ordenado = df.sort_values(by='a', ascending=False)
dfb_ordenado = df.sort_values(by='b', ascending=False)
dfc_ordenado = df.sort_values(by='c', ascending=False)

dfc_ordenado.loc[(dfc_ordenado['itens'] == 481)]
#dfa_ordenado.head(5)
#dfa_ordenado.loc[(dfb_ordenado['itens'] > 7) & (dfb_ordenado['itens'] < 50)].head(50)

,itens,a,b,c,d
480,481,-20.262073,0.074802,0.25001,0.499996


In [ ]:
 print("idxmaisa: ", idx[570])
print("idxmaisb: ", idx[628])
print("idxmaisc:", idx[480])

idxmaisa:  4181
idxmaisb:  622
idxmaisc: 5397


In [ ]:
idx_maisdificeis = np.array(dfb_ordenado['itens'][:30])
idx_maisdiscriminantes = np.array(dfa_ordenado['itens'][:30])
idx_chutes = np.array(dfc_ordenado['itens'][:30])

dfidx = pd.DataFrame({ 'mais dificeis': idx_maisdificeis,
                      'mais discriminantes': idx_maisdiscriminantes,
                      'chutes': idx_chutes
    
})

print(dfidx)
dfidx.to_csv('/content/idx.csv', index=False)

   mais dificeis  mais discriminantes  chutes
0             12                    6     124
1              9                   28     147
2             97                   48     175
3             91                   72      12
4             78                   25      90
5            183                   77     200
6            175                   93     114
7             20                   17     171
8            186                    3     130
9             55                   20     122


In [ ]:
idx_maisdificeis

array([ 6, 28, 48, 72, 25, 77, 93, 17,  3, 20])